## Hybrid Search Setup 

### Create the Table

In [ ]:
%%sql
CREATE TABLE vecs(
    id BIGINT(20), 
    url TEXT DEFAULT NULL,
    paragraph TEXT DEFAULT NULL, 
    v VECTOR(1536) NOT NULL,
    SHARD KEY(id), KEY(id) USING HASH
);

### Create Pipeline 

In [ ]:
%%sql
CREATE OR REPLACE PIPELINE wiki_pipeline AS
load data S3
's3://wikipedia-video-game-data/video-game-embeddings(1).csv'
config '{"region":"us-west-1"}'
credentials '{"aws_access_key_id": "", "aws_secret_access_key": ""}'
SKIP DUPLICATE KEY ERRORS
INTO TABLE vecs
FORMAT csv
FIELDS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\r\n';

In [ ]:
%%sql
START PIPELINE wiki_pipeline;

In [ ]:
%%sql
select count(*) from vecs;

### Adding Indexes to Facilitate Hybrid Search

In [ ]:
%%sql
ALTER TABLE vecs ADD FULLTEXT ft_para(paragraph);

In [ ]:
%%sql

ALTER TABLE vecs ADD VECTOR INDEX ivf_v(v)
    INDEX_OPTIONS '{"index_type":"IVF_FLAT"}';

In [ ]:
%%sql

OPTIMIZE TABLE vecs FULL;

### Setting New Search Vector 

In [ ]:
%%sql
SET @v_mario_kart = (SELECT v FROM vecs 
          WHERE URL = "https://en.wikipedia.org/wiki/Super_Mario_Kart" 
          ORDER BY id LIMIT 1);

### Hybrid Search

In [ ]:
%%sql

WITH full_text_search AS (
  SELECT id, paragraph, 
    MATCH(paragraph) AGAINST("mario kart") AS SCORE
  FROM vecs
  WHERE MATCH(paragraph) AGAINST("mario kart")
  ORDER BY SCORE desc 
  LIMIT 200
),
vector_search AS (
  SELECT id, paragraph, v <*> @v_mario_kart AS SCORE
  FROM vecs
  ORDER BY score DESC 
  LIMIT 200
)
SELECT vector_search.id, SUBSTRING(vector_search.paragraph,0,25), 
      FORMAT(IFNULL(full_text_search.score, 0) * .3 
             + IFNULL(vector_search.score, 0) * .7, 4) AS Score, 
      FORMAT(full_text_search.score, 4) AS Full_Text_Search, 
      FORMAT(vector_search.score, 4) AS Vector_Search_Score
FROM full_text_search FULL OUTER JOIN vector_search ON full_text_search.id = vector_search.id 
ORDER BY score DESC
LIMIT 5;